In [1]:
from gpn.data import Genome
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

/accounts/projects/yss/gbenegas/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
genome = Genome("../../results/genome.fa.gz")
genome

KeyboardInterrupt: 

In [ ]:
V = pd.read_csv("v2.0/credible_set.txt", sep="\t", dtype={"chr": str})
V

In [ ]:
V = V.groupby(["chr", "bp", "ea", "nea"]).susie.max().reset_index().rename(columns={
    "chr": "chrom", "bp": "pos", "ea": "alt", "nea": "ref", "susie": "pip"
})
V

In [ ]:
V = V[V.pip != -1]
V

In [ ]:
(V.pip > 0.99).sum(), (V.pip > 0.95).sum(), (V.pip > 0.90).sum(), (V.pip < 0.01).sum()

In [61]:
V = V[(V.pip > 0.9) | (V.pip < 0.01)]
V

,chrom,pos,alt,ref,pip
3,1,662622,A,G,4.651720e-12
4,1,693731,G,A,5.534810e-11
5,1,693823,C,G,3.978040e-12
6,1,706368,G,A,8.349990e-13
8,1,731718,C,T,7.095540e-10
...,...,...,...,...,...
1163071,9,141088356,G,C,6.645120e-03
1163072,9,141088413,C,G,6.022440e-03
1163073,9,141088628,C,A,6.972080e-03
1163074,9,141088641,G,T,8.413570e-03


In [62]:
from liftover import get_lifter


def lift_hg19_to_hg38(V):
    converter = get_lifter('hg19', 'hg38')

    def get_new_pos(v):
        try:
            res = converter[v.chrom][v.pos]
            assert len(res) == 1
            chrom, pos, strand = res[0]
            assert chrom.replace("chr", "")==v.chrom
            return pos
        except:
            return -1

    V.pos = V.apply(get_new_pos, axis=1)
    return V

In [63]:
V = lift_hg19_to_hg38(V)
V

/tmp/ipykernel_2811277/167421979.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  V.pos = V.apply(get_new_pos, axis=1)


,chrom,pos,alt,ref,pip
3,1,727242,A,G,4.651720e-12
4,1,758351,G,A,5.534810e-11
5,1,758443,C,G,3.978040e-12
6,1,770988,G,A,8.349990e-13
8,1,796338,C,T,7.095540e-10
...,...,...,...,...,...
1163071,9,138197906,G,C,6.645120e-03
1163072,9,138197963,C,G,6.022440e-03
1163073,9,138198178,C,A,6.972080e-03
1163074,9,138198191,G,T,8.413570e-03


In [64]:
V = V[["chrom", "pos", "ref", "alt", "pip"]]
V

,chrom,pos,ref,alt,pip
3,1,727242,G,A,4.651720e-12
4,1,758351,A,G,5.534810e-11
5,1,758443,G,C,3.978040e-12
6,1,770988,A,G,8.349990e-13
8,1,796338,T,C,7.095540e-10
...,...,...,...,...,...
1163071,9,138197906,C,G,6.645120e-03
1163072,9,138197963,G,C,6.022440e-03
1163073,9,138198178,A,C,6.972080e-03
1163074,9,138198191,T,G,8.413570e-03


In [65]:
V = V[(V.ref.str.len() == 1) & (V.alt.str.len() == 1)]
V

,chrom,pos,ref,alt,pip
3,1,727242,G,A,4.651720e-12
4,1,758351,A,G,5.534810e-11
5,1,758443,G,C,3.978040e-12
6,1,770988,A,G,8.349990e-13
8,1,796338,T,C,7.095540e-10
...,...,...,...,...,...
1163071,9,138197906,C,G,6.645120e-03
1163072,9,138197963,G,C,6.022440e-03
1163073,9,138198178,A,C,6.972080e-03
1163074,9,138198191,T,G,8.413570e-03


In [66]:
def find_ref_alt(v):
    ref_nuc = genome.get_nuc(v.chrom, v.pos).upper()
    if v.ref == ref_nuc:
        return v.ref, v.alt
    elif v.alt == ref_nuc:
        return v.alt, v.ref
    else:
        return None, None

In [67]:
V[["ref", "alt"]] = V.progress_apply(find_ref_alt, axis=1, result_type="expand")
V

100%|██████████| 758430/758430 [00:25<00:00, 29807.75it/s]
/tmp/ipykernel_2811277/2149238536.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  V[["ref", "alt"]] = V.progress_apply(find_ref_alt, axis=1, result_type="expand")


,chrom,pos,ref,alt,pip
3,1,727242,G,A,4.651720e-12
4,1,758351,A,G,5.534810e-11
5,1,758443,G,C,3.978040e-12
6,1,770988,A,G,8.349990e-13
8,1,796338,T,C,7.095540e-10
...,...,...,...,...,...
1163071,9,138197906,C,G,6.645120e-03
1163072,9,138197963,G,C,6.022440e-03
1163073,9,138198178,A,C,6.972080e-03
1163074,9,138198191,T,G,8.413570e-03


In [68]:
V = V.dropna(subset="ref")
V

,chrom,pos,ref,alt,pip
3,1,727242,G,A,4.651720e-12
4,1,758351,A,G,5.534810e-11
5,1,758443,G,C,3.978040e-12
6,1,770988,A,G,8.349990e-13
8,1,796338,T,C,7.095540e-10
...,...,...,...,...,...
1163071,9,138197906,C,G,6.645120e-03
1163072,9,138197963,G,C,6.022440e-03
1163073,9,138198178,A,C,6.972080e-03
1163074,9,138198191,T,G,8.413570e-03


In [72]:
def sort_chrom_pos(V):
    chrom_order = [str(i) for i in range(1, 23)] + ['X', 'Y']
    V.chrom = pd.Categorical(V.chrom, categories=chrom_order, ordered=True)
    V = V.sort_values(['chrom', 'pos'])
    V.chrom = V.chrom.astype(str)
    return V

V = sort_chrom_pos(V)
V

/tmp/ipykernel_2811277/2949986566.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  V.chrom = pd.Categorical(V.chrom, categories=chrom_order, ordered=True)


,chrom,pos,ref,alt,pip
3,1,727242,G,A,4.651720e-12
4,1,758351,A,G,5.534810e-11
5,1,758443,G,C,3.978040e-12
6,1,770988,A,G,8.349990e-13
8,1,796338,T,C,7.095540e-10
...,...,...,...,...,...
640454,22,50783303,T,C,0.000000e+00
640456,22,50784338,G,A,2.306390e-03
640458,22,50791228,G,A,0.000000e+00
640459,22,50791377,T,C,0.000000e+00


In [73]:
!mkdir -p ../../results/causaldb
V.to_parquet("../../results/causaldb/test.parquet", index=False)

/scratch/users/gbenegas/software/mambaforge/envs/gpn/lib/python3.11/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
